# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [4]:
from keras.src.models import Sequential
from keras.src.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image
import tensorflow as tf

In [5]:
# Verificar se há GPUs disponíveis
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"GPUs disponíveis: {gpus}")
else:
    print("Nenhuma GPU disponível.")

Nenhuma GPU disponível.


In [6]:
# Inicializa a CNN
classfier = Sequential()

# Convolução
classfier.add(Conv2D(32, (3, 3), input_shape=(256, 124, 1), activation='relu')) 

# Pooling
classfier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening
classfier.add(Flatten())

# Conexão completa
classfier.add(Dense(units=128, activation='relu'))
classfier.add(Dense(units=1, activation='sigmoid'))

# Compilando a rede com class_weight
classfier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# Aumentando dados
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# Assegurando que a proporção de classes seja considerada
training_set = train_datagen.flow_from_directory(
    'data/mri/train',
    target_size=(256, 124),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale'
)

test_set = test_datagen.flow_from_directory(
    'data/mri/validation',
    target_size=(256, 124),
    batch_size=32,
    class_mode='binary',
    color_mode='grayscale'
)

Found 18656 images belonging to 2 classes.
Found 3292 images belonging to 2 classes.


In [8]:
# Treinamento da rede
classfier.fit(
    training_set,
    steps_per_epoch=len(training_set),
    epochs=6,
    validation_data=test_set,
    validation_steps=len(test_set)
)

Epoch 1/6
583/583 [==============================] - 123s 210ms/step - loss: 0.4863 - accuracy: 0.7767 - val_loss: 0.3557 - val_accuracy: 0.8499
Epoch 2/6
583/583 [==============================] - 144s 247ms/step - loss: 0.3081 - accuracy: 0.8713 - val_loss: 0.2844 - val_accuracy: 0.8949
Epoch 3/6
583/583 [==============================] - 144s 248ms/step - loss: 0.2635 - accuracy: 0.8916 - val_loss: 0.5666 - val_accuracy: 0.7783
Epoch 4/6
583/583 [==============================] - 146s 250ms/step - loss: 0.2375 - accuracy: 0.9042 - val_loss: 0.6363 - val_accuracy: 0.7725
Epoch 5/6
583/583 [==============================] - 147s 252ms/step - loss: 0.2142 - accuracy: 0.9129 - val_loss: 0.1970 - val_accuracy: 0.9271
Epoch 6/6
583/583 [==============================] - 147s 252ms/step - loss: 0.1978 - accuracy: 0.9207 - val_loss: 0.1983 - val_accuracy: 0.9189


In [ ]:
# Teste
test_image = image.load_img('data/mri/HC-3051-15313-2588.png', color_mode='grayscale', target_size=(256, 124))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0) / 255.0  # Normalização

result = classfier.predict(test_image)
print(result)

if result[0][0] >= 0.5:
    prediction = 'Not Alzheimer'
else:
    prediction = 'Alzheimer'
print(prediction)